# MapD Charting Example with Altair

Let's see if we can replicate [this](https://omnisci.github.io/mapd-charting/example/example1.html) MapD charting example in Python with Altair, Vega Lite, and Vega:

![](https://cloud.githubusercontent.com/assets/2932405/25641647/1acce1f2-2f4a-11e7-87d4-a4e80cb262f5.gif)

In [1]:
import altair as alt
import jupyterlab_omnisci.vega_ibis
import ibis

alt.data_transformers.enable('ibis')
alt.renderers.enable('ibis')

RendererRegistry.enable('ibis')

In [2]:
conn = ibis.mapd.connect(
    host='metis.mapd.com', user='mapd', password='HyperInteractive',
    port=443, database='mapd', protocol= 'https'
)
t = conn.table("flights_donotmodify")

In [3]:
t.info()

Table rows: 7009728

Column    Type      Non-null #
------    ----      ----------
flight_yearint16     7009728   
flight_monthint16     7009728   
flight_dayofmonthint16     7009728   
flight_dayofweekint16     7009728   
deptime   int16     6873482   
crsdeptimeint16     7009728   
arrtime   int16     6858079   
crsarrtimeint16     7009728   
uniquecarrierString(nullable=True)7009728   
flightnum int16     7009728   
tailnum   String(nullable=True)6926363   
actualelapsedtimeint16     6855029   
crselapsedtimeint16     7008884   
airtime   int16     6855029   
arrdelay  int16     6855029   
depdelay  int16     6873482   
origin    String(nullable=True)7009728   
dest      String(nullable=True)7009728   
distance  int16     7009728   
taxiin    int16     6858079   
taxiout   int16     6872670   
cancelled int16     7009728   
cancellationcodeString(nullable=True)137434    
diverted  int16     7009728   
carrierdelayint16     1524735   
weatherdelayint16     1524735   
nasdelay  int16 

In [8]:
jupyterlab_omnisci.vega_ibis.FALLBACK = False

alt.Chart(t[t.dep_timestamp, t.depdelay]).mark_point().encode(
    x='yearmonthdate(dep_timestamp):O',
    y='mean(depdelay):Q'
)

Chart({
  data: Empty DataFrame
  Columns: [dep_timestamp, depdelay]
  Index: [],
  encoding: FacetedEncoding({
    x: X({
      field: 'dep_timestamp',
      timeUnit: 'yearmonthdate',
      type: 'ordinal'
    }),
    y: Y({
      aggregate: 'mean',
      field: 'depdelay',
      type: 'quantitative'
    })
  }),
  mark: 'point'
})

In [ ]:
states = alt.selection_multi(fields=['origin_state'])
airlines = alt.selection_multi(fields=['carrier_name'])

flights_by_state = alt.Chart(
    t[t.origin_state, t.carrier_name],
    title="Total Number of Flights by State"
).transform_filter(
    airlines
).mark_bar().encode(
    x='count()',
    y=alt.Y('origin_state', sort=alt.Sort(encoding='x', order='descending')),
    color=alt.condition(states, alt.ColorValue("steelblue"), alt.ColorValue("grey"))
).add_selection(states).properties(
    height=400,
    width=600
)


carrier_delay = alt.Chart(
    t[t.depdelay, t.arrdelay, t.carrier_name, t.origin_state],
    title="Carrier Departure Delay by Arrival Delay (Minutes)"
).transform_filter(
    states
).transform_aggregate(
    depdelay='mean(depdelay)',
    arrdelay='mean(arrdelay)',
    groupby=["carrier_name"]
).mark_point(filled=True, size=200).encode(
    x='depdelay',
    y='arrdelay',
    color=alt.condition(airlines, alt.ColorValue("steelblue"), alt.ColorValue("grey")),
    tooltip=['carrier_name', 'depdelay', 'arrdelay']
).add_selection(airlines).properties(
    height=400,
    width=600
)

# jupyterlab_omnisci.vega_ibis.FALLBACK = False

(flights_by_state | carrier_delay)

In [14]:
jupyterlab_omnisci.vega_ibis.FALLBACK = True

# (flights_by_state | carrier_delay)